# Fwumious Wabbit demo

 to learn more about Fwumious Wabbit go [here]( https://github.com/outbrain/fwumious_wabbit)

## Wabbit showdown: Fwumious vs. Vowpal


In [12]:
!python print_system_info.py

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Current Frequency: 2900.00Mhz
======================================== System Information ========================================
System: Darwin
Release: 19.6.0
Version: Darwin Kernel Version 19.6.0: Thu Jun 18 20:49:00 PDT 2020; root:xnu-6153.141.1~1/RELEASE_X86_64
Machine: x86_64
Processor: i386


### the task: predict which animal likes to eat what


We generate a dataset containing 5,000,000 simple combinations of animal, food and outcome - either the animal likes the food (1) or it dislikes it (-1)

Namespace A will be Animal, and B will be Food.

Each animal has its latent type - Herbivore or Carnivore.

Each food has its latent type - Plant or Meat.

We use feature names like "Herbivore-13" and "Plant-55". for vw and fw, these are just strings. But they give us humans a hint as to the expected outcome (herbivores like plants, and carnivores like meat...)

### simple logistic regression showdown



**genereate dataset**

In [ ]:
!python cleanup.py

In [19]:
!python generate.py

review the first 10 lines of the generated training data

In [24]:
!head -n 10 train.vw

-1 |A Carnivore-0 |B Plant-2
1 |A Carnivore-2 |B Meat-4
-1 |A Carnivore-1 |B Plant-2
-1 |A Carnivore-2 |B Plant-4
-1 |A Carnivore-5 |B Plant-0
1 |A Carnivore-5 |B Meat-3
1 |A Carnivore-0 |B Meat-1
-1 |A Carnivore-1 |B Plant-4
1 |A Herbivore-2 |B Plant-3
-1 |A Herbivore-3 |B Meat-1


In [42]:
!gzip train.vw

**measure vw training time - not reading from cache (cache file is created)**

*note that in order for the model to learn successfully - we use feature interactions. if we replace "--interactions AB" with "--keep A --keep B" the model won't improve with training.*

In [43]:
!python clean_caches.py

In [44]:
!(time vw --data train.vw.gz -l 0.1 -b 25 -c --adaptive --sgd --loss_function logistic --link logistic --power_t 0.0 --l2 0.0 --hash all --final_regressor vw_model --save_resume --interactions AB) 2>&1 >/dev/null  | tail -n 1 | awk -F" " '{ print $13" "$14" "$15" "$16" "$17" "$18" "$19" "$20}'

13.27s user 6.79s system 141% cpu 14.196 total


**measure fw training time - not reading from cache (cache file is created)**

In [45]:
!(time ./fw --data train.vw.gz -l 0.1 -b 25 -c --adaptive --fastmath --sgd --loss_function logistic --link logistic --power_t 0.0 --l2 0.0 --hash all --final_regressor fw_model --save_resume --interactions AB) 2>&1 >/dev/null  | tail -n 1 | awk -F" " '{ print $13" "$14" "$15" "$16" "$17" "$18" "$19" "$20}'

2.82s user 0.34s system 97% cpu 3.248 total


**now repeat the training for vw and fw - this time using the cache:**

In [46]:
!(time vw --data train.vw.gz -l 0.1 -b 25 -c --adaptive --sgd --loss_function logistic --link logistic --power_t 0.0 --l2 0.0 --hash all --interactions AB) 2>&1 >/dev/null  | tail -n 1 | awk -F" " '{ print $13" "$14" "$15" "$16" "$17" "$18" "$19" "$20}'

7.35s user 6.53s system 157% cpu 8.800 total


In [47]:
!(time ./fw --data train.vw.gz -l 0.1 -b 25 -c --adaptive --sgd --loss_function logistic --link logistic --power_t 0.0 --l2 0.0 --hash all --interactions AB) 2>&1 >/dev/null  | tail -n 1 | awk -F" " '{ print $13" "$14" "$15" "$16" "$17" "$18" "$19" "$20}'

user 0.15s system 95% cpu 1.216 total 


**make predictions with both models:**